## Loading Necessary Libraries

In [1]:
# general
import os
import re
import gc
import csv
import time
import pickle
from tqdm import tqdm

# data handling
import numpy as np
import pandas as pd

# HTML parsing
from bs4 import BeautifulSoup

# plotting
import matplotlib.pyplot as plt

# chunker
import torch
from torch import mps
import flair
from flair.data import Sentence
from flair.models import SequenceTagger

## Load Data

In [2]:
# 2023 Data
# data = pd.read_csv('../data/2023/train-v3.csv')
# data = pd.read_csv('../data/2023/test-v3.csv')

# 2021 Data
# data = pd.read_csv('../data/2021/train-v3.csv')
data = pd.read_csv('../data/2021/test-v3.csv')

/var/folders/mg/l90d6s9536n6mfzjbqgrxffm0000gn/T/ipykernel_7541/3678938305.py:6: DtypeWarning: Columns (21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../data/2021/train-v3.csv')


In [3]:
data = data.fillna('')

In [4]:
data

,TITLE,DESCRIPTION,BRAND,BROWSE_NODE_ID,0,1,2,3,4,5,...,214,215,216,217,218,219,220,221,222,223
0,"Pete The Cat Bedtime Blues Doll , 14.5 Inch","Pete the Cat is the coolest , most popular cat...",MerryMakers,0,Pete the Cat Bedtime Blues plush doll,Based on the popular Pete the Cat books by Jam...,Super cuddly and ready for naptime and bedtime,"Safe for all ages , perfect for ages 2 and up",Measures 14.5 inches,,...,,,,,,,,,,
1,"The New Yorker NYHM014 Refrigerator Magnet , 2...",The New Yorker Handsome Cello Wrapped Hard Mag...,The New Yorker,1,Cat In A Tea Cup by New Yorker cover artist Gu...,Handsome cello wrapped hard magnet,"Ideal for home or office , or for a gift and f...",It highlight one of the many beautiful New Yor...,Rigid magnet measures 2.5-Inch width by 3.5-In...,,...,,,,,,,,,,
2,The Ultimate Self-Sufficiency Handbook : A Com...,,imusti,2,Skyhorse Publishing,,,,,,...,,,,,,,,,,
3,Amway Nutrilite Kids Chewable Iron Tablets ( 1...,,Amway,3,Nutrilite Kids,Chewable Iron Tablets,Quantity 100 tablets,,,,...,,,,,,,,,,
4,Teacher Planner Company A4 6 Lesson Academic T...,,,4,,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2903019,Premium Aviator SunglassesHD Polarized ( Brigh...,These premium Aviator Sunglasses with 5 color ...,Generic,1040,"Frame size : Lens height56mm , Lens width58 mm...",Unisex Sunglasses,Comes in 5 colors,Sunglasses come with a premium case to keep th...,Lenses are made with TAC material,,...,,,,,,,,,,
2903020,Social Distance StickersSet of 5 Sticker Slip ...,set of 5 prints social distancing sticker self...,Generic,15199,covid19 safety stickerset of 5 to maintain cro...,"easy to paste on wall , tiles , marble , plast...",made from self adhesive pvc durable vinyl mate...,can be used in any placerestaurant or offices ...,,,...,,,,,,,,,,
2903021,Torr-to Face Shield PACK OF 5 with Adjustable ...,* COMPLETE FACE PROTECTION : Torr-to Face Shie...,TORR-TO,1044933,350 MICRONS PACK OF 5 pieces,COMPLETE FACE PROTECTION : Torr-to Face Shield...,"COMFORTABLE , ADJUSTABLE and REUSABLE : Equipp...",ANTI-FOG and ANTI-STATIC : It is coated with a...,FITS FOR ALL : Easy to adjust for a custom fit...,,...,,,,,,,,,,
2903022,Type-C to 3.5 MM for Oppo R17 Pro Type-C to 3....,Still want to use your favorite earphones or h...,SHOPBELL,14790,"Indian Connectors : Made for Indian sockets , ...",Smart Charge : Auto Detects the connected devi...,EXCLUSIVE TECHNOLOGY : India ' s smallest char...,Fire-resistant and hardened plastic outer shel...,Good Charging Speeds Most PowerfulCharge any U...,,...,,,,,,,,,,


## Chunking

Since, we have lot of text data, capturing the entire graph between all the tokens would be difficult (time consuming). So, let's chunk the text into sections to capture the local information.

In [5]:
flair.device = torch.device('cpu')

In [6]:
flair.device

device(type='cpu')

In [7]:
# load tagger
tagger = SequenceTagger.load("flair/chunk-english-fast")

2023-05-27 11:21:56,129 SequenceTagger predicts: Dictionary with 47 tags: O, S-NP, B-NP, E-NP, I-NP, S-VP, B-VP, E-VP, I-VP, S-PP, B-PP, E-PP, I-PP, S-ADVP, B-ADVP, E-ADVP, I-ADVP, S-SBAR, B-SBAR, E-SBAR, I-SBAR, S-ADJP, B-ADJP, E-ADJP, I-ADJP, S-PRT, B-PRT, E-PRT, I-PRT, S-CONJP, B-CONJP, E-CONJP, I-CONJP, S-INTJ, B-INTJ, E-INTJ, I-INTJ, S-LST, B-LST, E-LST, I-LST, S-UCP, B-UCP, E-UCP, I-UCP, <START>, <STOP>


In [120]:
data.DESCRIPTION[1]

'The New Yorker Handsome Cello Wrapped Hard Magnet measures 2.5-Inch width by 3.5-Inch height and highlight one of the many beautiful New Yorker covers in full color . Cat In A Tea Cup by New Yorker cover artist Gurbuz Dogan Eksioglu'

In [121]:
data.DESCRIPTION[1].split(' . ')

['The New Yorker Handsome Cello Wrapped Hard Magnet measures 2.5-Inch width by 3.5-Inch height and highlight one of the many beautiful New Yorker covers in full color',
 'Cat In A Tea Cup by New Yorker cover artist Gurbuz Dogan Eksioglu']

In [122]:
for i in range(1000):
  print()
  print(f'***{i}***')
  print()
  print('\n\n'.join(data.DESCRIPTION[i].split(' . ')))


***0***

Pete the Cat is the coolest , most popular cat in town

The new Pete the Cat Bedtime Blues Doll from MerryMakers rocks his striped pj ' s and red slippers

He is one sleepy cat and is ready to cuddle

Measures 14.5 inches tall

Safe for all ages

Removable clothing

Surface wash only

New 2015

***1***

The New Yorker Handsome Cello Wrapped Hard Magnet measures 2.5-Inch width by 3.5-Inch height and highlight one of the many beautiful New Yorker covers in full color

Cat In A Tea Cup by New Yorker cover artist Gurbuz Dogan Eksioglu

***2***



***3***



***4***



***5***

Men ' S Full Sleeve Raglan T-Shirts Denim T-Shirt M Size Only

***6***

This Black wallet by Glance will be a treasured addition to your accessory collection

Featuring an attractive design , this wallet will definitely lend you a sophisticated look

It is crafted from durable Artificial Leather material and has multiple slots to accommodate essentials like identity card , cash , credit or debit cards and o

In [127]:
data.TITLE[0]

'Pete The Cat Bedtime Blues Doll , 14.5 Inch'

In [124]:
# make example sentence
sentences = [Sentence(data.TITLE.to_list()[0])]

# predict NER tags
tagger.predict(sentences)

# print predicted NER spans
for sentence in sentences:
  print()
  print('The following NER tags are found:')
  # iterate over entities and print
  for entity in sentence.get_spans('np'):
      print(entity)


The following NER tags are found:
Span[0:6]: "Pete The Cat Bedtime Blues Doll" → NP (0.8273)
Span[7:9]: "14.5 Inch" → NP (0.9688)


In [128]:
# make example sentence
sentences = [Sentence(sent) for sent in data.DESCRIPTION[0].split(' . ')]

# predict NER tags
tagger.predict(sentences)

# print predicted NER spans
for sentence in sentences:
  print()
  print('The following NER tags are found:')
  # iterate over entities and print
  for entity in sentence.get_spans('np'):
      print(entity)


The following NER tags are found:
Span[1:3]: "the Cat" → NP (0.7455)
Span[3:4]: "is" → VP (1.0)
Span[4:10]: "the coolest , most popular cat" → NP (0.9063)
Span[10:11]: "in" → PP (1.0)
Span[11:12]: "town" → NP (1.0)

The following NER tags are found:
Span[0:8]: "The new Pete the Cat Bedtime Blues Doll" → NP (0.905)
Span[8:9]: "from" → PP (1.0)
Span[9:10]: "MerryMakers" → NP (0.9999)
Span[10:11]: "rocks" → VP (0.9999)
Span[11:19]: "his striped pj ' s and red slippers" → NP (0.8644)

The following NER tags are found:
Span[0:1]: "He" → NP (1.0)
Span[1:2]: "is" → VP (1.0)
Span[2:5]: "one sleepy cat" → NP (0.8752)
Span[6:7]: "is" → VP (0.9987)
Span[7:8]: "ready" → ADJP (0.9999)
Span[8:10]: "to cuddle" → VP (0.9469)

The following NER tags are found:
Span[0:1]: "Measures" → NP (0.9977)
Span[1:3]: "14.5 inches" → NP (0.751)
Span[3:4]: "tall" → ADJP (0.9904)

The following NER tags are found:
Span[0:1]: "Safe" → ADJP (0.9741)
Span[1:2]: "for" → PP (1.0)
Span[2:4]: "all ages" → NP (0.9834)

The

In [130]:
data.iloc[:,1]

0          Pete the Cat is the coolest , most popular cat...
1          The New Yorker Handsome Cello Wrapped Hard Mag...
2                                                           
3                                                           
4                                                           
                                 ...                        
2903019    These premium Aviator Sunglasses with 5 color ...
2903020    set of 5 prints social distancing sticker self...
2903021    * COMPLETE FACE PROTECTION : Torr-to Face Shie...
2903022    Still want to use your favorite earphones or h...
2903023    Performance inspired by comfort . The polo shi...
Name: DESCRIPTION, Length: 2903024, dtype: object

In [131]:
(data.shape[0] // PRODUCT_AT_ONCE)

145

In [8]:
file_no = 1
PRODUCT_AT_ONCE = 20000

for i in tqdm(range((data.shape[0] // PRODUCT_AT_ONCE) + 1)):
  # make product sentences
  sentences = [Sentence(data.iloc[k,0]) for k in range(i*PRODUCT_AT_ONCE, min((i+1)*PRODUCT_AT_ONCE, data.shape[0]))]
  
  # predict chunk tags
  tagger.predict(sentences, verbose=True, mini_batch_size=1000)
  
  processed_chunks = list()
  
  # print predicted chunk spans
  for sentence in sentences:
    
    processed_sent = list()
    
    # iterate over entities and print
    for entity in sentence.get_spans('np'):
      processed_sent.append((entity.text, entity.tag))
    
    processed_chunks.append(processed_sent)
    
  with open(f'../data/2021/temp-titles/train-{file_no}.pkl', 'wb') as f:
    pickle.dump(processed_chunks, f)

  file_no += 1

  mps.empty_cache()
  gc.collect()

  0%|                                                   | 0/146 [00:00<?, ?it/s]

2023-05-27 11:22:28,717 Warning: An empty Sentence was created! Are there empty strings in your dataset?



  1%|▎                                       | 1/146 [02:10<5:14:36, 130.18s/it]

2023-05-27 11:24:39,542 Warning: An empty Sentence was created! Are there empty strings in your dataset?



  1%|▌                                       | 2/146 [04:20<5:12:28, 130.20s/it]

...


 96%|████████████████████████████████████▍ | 140/146 [5:11:11<12:53, 128.92s/it]

2023-05-27 16:33:39,843 Warning: An empty Sentence was created! Are there empty strings in your dataset?



 97%|████████████████████████████████████▋ | 141/146 [5:13:18<10:42, 128.56s/it]

2023-05-27 16:35:47,165 Warning: An empty Sentence was created! Are there empty strings in your dataset?



100%|██████████████████████████████████████| 146/146 [5:22:10<00:00, 132.40s/it]


In [9]:
os.listdir('../data/2021/temp-titles')

['train-98.pkl',
 'train-73.pkl',
 'train-129.pkl',
 'train-67.pkl',
 'train-115.pkl',
 'train-101.pkl',
 'train-100.pkl',
 'train-114.pkl',
 'train-66.pkl',
 'train-72.pkl',
 'train-128.pkl',
 'train-99.pkl',
 'train-64.pkl',
 'train-70.pkl',
 'train-58.pkl',
 'train-102.pkl',
 'train-116.pkl',
 'train-117.pkl',
 'train-59.pkl',
 'train-103.pkl',
 'train-71.pkl',
 'train-65.pkl',
 'train-107.pkl',
 'train-49.pkl',
 'train-113.pkl',
 'train-61.pkl',
 'train-75.pkl',
 '.DS_Store',
 'train-74.pkl',
 'train-60.pkl',
 'train-48.pkl',
 'train-112.pkl',
 'train-106.pkl',
 'train-89.pkl',
 'train-110.pkl',
 'train-104.pkl',
 'train-76.pkl',
 'train-62.pkl',
 'train-138.pkl',
 'train-63.pkl',
 'train-139.pkl',
 'train-77.pkl',
 'train-105.pkl',
 'train-111.pkl',
 'train-88.pkl',
 'train-10.pkl',
 'train-38.pkl',
 'train-39.pkl',
 'train-11.pkl',
 'train-13.pkl',
 'train-12.pkl',
 'train-9.pkl',
 'train-16.pkl',
 'train-17.pkl',
 'train-8.pkl',
 'train-29.pkl',
 'train-15.pkl',
 'train-14.pkl',

In [10]:
i

145

In [11]:
chunked_titles = [pickle.load(open(f'../data/2021/temp-titles/train-{k+1}.pkl', 'rb')) for k in tqdm(range(i+1))]

100%|█████████████████████████████████████████| 146/146 [00:09<00:00, 15.64it/s]


In [12]:
chunked_titles = [chunk for chunk_title in tqdm(chunked_titles) for chunk in chunk_title]

100%|████████████████████████████████████████| 146/146 [00:00<00:00, 800.38it/s]


In [13]:
np.mean([len(chunk) for chunk in chunked_titles])

5.0520936099736

In [16]:
data.TITLE.to_list()[:10]

['Pete The Cat Bedtime Blues Doll , 14.5 Inch',
 'The New Yorker NYHM014 Refrigerator Magnet , 2 x 3.5',
 'The Ultimate Self-Sufficiency Handbook : A Complete Guide to Baking , Crafts , Gardening , Preserving Your Harvest , Raising Animals and More',
 'Amway Nutrilite Kids Chewable Iron Tablets ( 100 )',
 'Teacher Planner Company A4 6 Lesson Academic Teacher PlannerLeatherette Pink',
 "Men ' S Full Sleeve Raglan T-Shirts Denim T-Shirt M Size Only",
 "Glance Women ' s Wallet ( Black ) ( LW-21 )",
 'Wild Animals Hungry Brain Educational Flash Cards for Enhancing Early Learning of Your Kids ( 3 Month to 6 Year )',
 'Hindi Barakhadi Educational Flash Cards for Kids Enhance Early Learning of Your Kids ( 3 Months to 6 Years )',
 'Chapin 1380 Premier Pro 3-Gallon Tri-Poxy Steel Sprayer for Fertilizer']

In [14]:
chunked_titles[:10]

[[('Pete The Cat Bedtime Blues Doll', 'NP'), ('14.5 Inch', 'NP')],
 [('The New Yorker NYHM014 Refrigerator Magnet', 'NP'), ('2 x', 'NP')],
 [('The Ultimate Self-Sufficiency Handbook', 'NP'),
  ('A Complete Guide', 'NP'),
  ('to', 'PP'),
  ('Baking', 'NP'),
  ('Crafts', 'NP'),
  ('Gardening', 'NP'),
  ('Preserving', 'VP'),
  ('Your Harvest', 'NP'),
  ('Raising', 'VP'),
  ('Animals', 'NP'),
  ('More', 'NP')],
 [('Amway Nutrilite Kids Chewable Iron Tablets', 'NP'), ('100', 'NP')],
 [('Teacher Planner Company A4 6 Lesson Academic Teacher PlannerLeatherette Pink',
   'NP')],
 [('Men', 'NP'),
  ("' S Full Sleeve Raglan T-Shirts", 'NP'),
  ('Denim', 'VP'),
  ('T-Shirt M Size Only', 'NP')],
 [('Glance Women', 'NP'),
  ("' s Wallet", 'NP'),
  ('Black )', 'NP'),
  ('LW-21', 'NP')],
 [('Wild Animals Hungry Brain Educational Flash Cards', 'NP'),
  ('for', 'PP'),
  ('Enhancing', 'VP'),
  ('Early Learning', 'NP'),
  ('of', 'PP'),
  ('Your Kids', 'NP'),
  ('3 Month', 'NP'),
  ('6 Year', 'NP')],
 [('H

In [17]:
with open(f'../data/2021/temp-titles/chunked-train-title-2021.pkl', 'wb') as f:
  pickle.dump(chunked_titles, f)

In [18]:
len(chunked_titles)

2903024